# stacgee: Google Earth Engine with STAC Metadata Tutorial

**stacgee** (rewritten from `geestac`) provides a powerful way to interact with the Google Earth Engine STAC catalog and hundreds of other public STAC catalogs. It allows you to initialize GEE objects while retaining access to the rich metadata stored in the STAC catalog.

Author: Pulakesh Pradhan  
Repository: [https://github.com/pulakeshpradhan/stacgee](https://github.com/pulakeshpradhan/stacgee)

## 1. Installation and Setup

First, ensure you have the necessary libraries installed. You will need `earthengine-api`, `geemap`, and `stacgee`.

### Install from PyPI or GitHub
```bash
pip install earthengine-api stacgee geemap
```

### Install from locally built .whl file
If you have built the `.whl` file locally, you can install it as follows (adjust the path to your `dist` folder):

In [ ]:
# Example: Installing the locally built wheel
# !pip install ../dist/stacgee-0.0.1-py3-none-any.whl

In [ ]:
import ee
import stacgee
import geemap
import json

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='my-project')

## 2. Dynamic World: Accessing Land Cover Classes

Dynamic World is a 10m global land cover product. In standard GEE, finding the class names and colors requires checking the documentation. With `stacgee`, it's available directly in the object metadata.

In [ ]:
# Initialize like a standard GEE ImageCollection
dw = stacgee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

print(f"Asset ID: {dw.assetId}")
print(f"Start Date: {dw.start_date}")

# Access class information from the 'label' band
label_band = dw.bands.label
print("\nDynamic World Classes:")
for category in label_band.class_info:
    print(f"  Value {category.value}: {category.description} (Color: {category.color})")

## 3. Sentinel-2: Understanding QA Bitmasks

Quality assessment bands (like `QA60`) often store multiple flags in individual bits. `stacgee` decodes these bitmasks for you.

In [ ]:
s2 = stacgee.ImageCollection("COPERNICUS/S2_SR")
qa = s2.bands.QA60

if hasattr(qa, "bitmask"):
    print("Sentinel-2 QA60 Bitmask Definition:")
    print(json.dumps(qa.bitmask.to_dict(), indent=2))

## 4. GEE Processing via Proxy

`stacgee` objects act as proxies to the underlying GEE objects. You can call native GEE methods like `.filter()`, `.median()`, or `.clip()` directly on them.

In [ ]:
# Create a region of interest
roi = ee.Geometry.Point([77.2090, 28.6139]).buffer(5000)

# filterDate and filterBounds are GEE methods!
# They work here because dw proxies to the underlying ImageCollection.
filtered_dw = dw.filterBounds(roi).filterDate('2023-01-01', '2023-06-01')

print(f"Images found: {filtered_dw.size().getInfo()}")

# Create a composite
median_composite = filtered_dw.select('label').mode()

# Visualize
Map = geemap.Map()
Map.centerObject(roi, 12)
Map.addLayer(median_composite, {'min': 0, 'max': 8, 'palette': ['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc351', 'c4281b', 'a59b8f', 'b39fe1']}, 'Dynamic World Mode')
Map

## 5. MODIS: Scaling and Offsets

Many GEE datasets require multiplying raw values by a scale factor. `stacgee` makes these factors easily accessible.

In [ ]:
modis = stacgee.ImageCollection("MODIS/061/MOD11A1")
lst_day = modis.bands.LST_Day_1km

print(f"Band: {lst_day.name}")
print(f"Scale Factor: {lst_day.multiplier}")
print(f"Offset: {lst_day.offset}")

# Applying the scale directly in the compute expression
actual_temp = modis.select('LST_Day_1km').first().multiply(lst_day.multiplier).add(lst_day.offset)
print("Successfully applied STAC-derived scale and offset to GEE Image.")

## 6. Accessing Global STAC Catalogs (v0.0.1+)

You can now access hundreds of public STAC catalogs via the **STAC Index** integration.

In [ ]:
from stacgee import stacindex

# List the first 10 experimental items in STAC Index
print("Available Global Catalogs (Sample):")
print(list(stacindex.children.keys())[:10])

# Access the Microsoft Planetary Computer
pc = stacindex.microsoft_pc()
print(f"\nCatalog: {pc.title}")
print(f"Description: {pc.description}")